# Transformers Examples

In [6]:
# !pip install transformers

# All the following examples can be found here:
# https://huggingface.co/transformers/task_summary.html

In [ ]:
from transformers import pipeline
from pprint import pprint

### Sequence Classification

In [14]:
nlp = pipeline("sentiment-analysis")

In [15]:
result = nlp("I hate you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: NEGATIVE, with score: 0.9991


In [16]:
result = nlp("I love you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9999


### Extractive Question Answering
Extractive Question Answering is the task of **extracting an answer from a text given a question**. An example of a question answering dataset is the SQuAD dataset, which is entirely based on that task.  

Here is an example of using pipelines to do question answering: extracting an answer from a text given a question. It leverages a fine-tuned model on SQuAD.

In [17]:
nlp = pipeline("question-answering")

In [18]:
context = r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.
"""

print(context)


Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/pytorch/question-answering/run_squad.py script.



This returns an answer extracted from the text, a confidence score, alongside “start” and “end” values, which are the positions of the extracted answer in the text.



In [19]:
result = nlp(question="What is extractive question answering?",
             context=context)


print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'the task of extracting an answer from a text given a question', score: 0.6177, start: 34, end: 95


In [20]:
result = nlp(question="What is a good example of a question answering dataset?", 
             context=context)

print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

Answer: 'SQuAD dataset', score: 0.5152, start: 147, end: 160


Here is an example of question answering using a model and a tokenizer. The process is the following:

1. Instantiate a tokenizer and a model from the checkpoint name. The model is identified as a BERT model and loads it with the weights stored in the checkpoint.

2. Define a text and a few questions.

3. Iterate over the questions and build a sequence from the text and the current question, with the correct model-specific separators token type ids and attention masks.

4. Pass this sequence through the model. This outputs a range of scores across the entire sequence tokens (question and text), for both the start and end positions.

5. Compute the softmax of the result to get probabilities over the tokens.

6. Fetch the tokens from the identified start and stop values, convert those tokens to a string.

7. Print the results.

In [21]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering
import tensorflow as tf

In [22]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [23]:
text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""

In [24]:
questions = [
    "How many pretrained models are available in 🤗 Transformers?",
    "What does 🤗 Transformers provide?",
    "🤗 Transformers provides interoperability between which frameworks?",
]

In [25]:
for question in questions:
    inputs = tokenizer(question, text, add_special_tokens=True, return_tensors="tf")

    input_ids = inputs["input_ids"].numpy()[0]
    
    outputs = model(inputs)
    
    answer_start_scores = outputs.start_logits
    
    answer_end_scores = outputs.end_logits
    
    # Get the most likely beginning of answer with the argmax of the score
    answer_start = tf.argmax(answer_start_scores, axis=1).numpy()[0]  
    
    # Get the most likely end of answer with the argmax of the score
    answer_end = (tf.argmax(answer_end_scores, axis=1) + 1).numpy()[0]

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print('--------------------------------------------------')

Question: How many pretrained models are available in 🤗 Transformers?
Answer: over 32 +
--------------------------------------------------
Question: What does 🤗 Transformers provide?
Answer: general - purpose architectures
--------------------------------------------------
Question: 🤗 Transformers provides interoperability between which frameworks?
Answer: tensorflow 2. 0 and pytorch
--------------------------------------------------


### Language Modeling

Language modeling is the task of fitting a model to a corpus, which can be domain specific. All popular transformer-based models are trained using a variant of language modeling, e.g. BERT with masked language modeling, GPT-2 with causal language modeling.

Language modeling can be useful outside of **pretraining** as well, for example to shift the model distribution to be domain-specific: using a language model **trained over a very large corpus**, and then **fine-tuning** it to a news dataset or on scientific papers.

#### Masked Language Modeling

Masked language modeling is the task of masking tokens in a sequence with a masking token, and prompting the model to fill that mask with an appropriate token. This allows the model to attend to both the right context (tokens on the right of the mask) and the left context (tokens on the left of the mask). Such a training creates a strong basis for downstream tasks requiring bi-directional context, such as SQuAD (question answering, see Lewis, Lui, Goyal et al., part 4.2). If you would like to fine-tune a model on a masked language modeling task, you may leverage the run_mlm.py script.

In [26]:
# Here is an example of using pipelines to replace a mask from a sequence:

nlp = pipeline("fill-mask")

In [27]:
# This outputs the sequences with the mask filled, the confidence score, and the token id in the tokenizer vocabulary:

text = f"HuggingFace is creating a {nlp.tokenizer.mask_token} that the community uses to solve NLP tasks."


pprint(nlp(text))

[{'score': 0.17927460372447968,
  'sequence': 'HuggingFace is creating a tool that the community uses to solve '
              'NLP tasks.',
  'token': 3944,
  'token_str': ' tool'},
 {'score': 0.1134939044713974,
  'sequence': 'HuggingFace is creating a framework that the community uses to '
              'solve NLP tasks.',
  'token': 7208,
  'token_str': ' framework'},
 {'score': 0.05243545398116112,
  'sequence': 'HuggingFace is creating a library that the community uses to '
              'solve NLP tasks.',
  'token': 5560,
  'token_str': ' library'},
 {'score': 0.03493543714284897,
  'sequence': 'HuggingFace is creating a database that the community uses to '
              'solve NLP tasks.',
  'token': 8503,
  'token_str': ' database'},
 {'score': 0.02860247902572155,
  'sequence': 'HuggingFace is creating a prototype that the community uses to '
              'solve NLP tasks.',
  'token': 17715,
  'token_str': ' prototype'}]


#### Text Generation

In text generation (a.k.a open-ended text generation) the goal is to create **a coherent portion of text** that is a continuation from the given context. The following example shows how GPT-2 can be used in pipelines to generate text. As a default all models apply Top-K sampling when used in pipelines, as configured in their respective configurations (see gpt-2 config for example).

In [28]:
text_generator = pipeline("text-generation")

In [29]:
pprint(text_generator("As far as I am concerned, I will",
                     max_length=50,
                     do_sample=False))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'As far as I am concerned, I will be the first to admit '
                    'that I am not a fan of the idea of a "free market." I '
                    'think that the idea of a free market is a bit of a '
                    'stretch. I think that the idea'}]


### Named Entity Recognition

Named Entity Recognition (NER) is the task of **classifying tokens according to a class**, for example, identifying a token as a person, an organisation or a location. An example of a named entity recognition dataset is the CoNLL-2003 dataset, which is entirely based on that task. If you would like to fine-tune a model on an NER task, you may leverage the run_ner.py script.

Here is an example of using pipelines to do named entity recognition, specifically, trying to identify tokens as belonging to one of 9 classes:

- O, Outside of a named entity
- B-MIS, Beginning of a miscellaneous entity right after another miscellaneous entity
- I-MIS, Miscellaneous entity
- B-PER, Beginning of a person’s name right after another person’s name
- I-PER, Person’s name
- B-ORG, Beginning of an organisation right after another organisation
- I-ORG, Organisation
- B-LOC, Beginning of a location right after another location
- I-LOC, Location

In [30]:
nlp = pipeline("ner")

In [31]:
sequence = """Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, 
therefore very close to the Manhattan Bridge which is visible from the window."""

print(sequence)

Hugging Face Inc. is a company based in New York City. Its headquarters are in DUMBO, 
therefore very close to the Manhattan Bridge which is visible from the window.


In [32]:
res = nlp(sequence)

for doc in res:
    print(f"Word: {doc['word']} | Score: {round(doc['score'], 3)} | Entity: {doc['entity']}")

Word: Hu | Score: 1.0 | Entity: I-ORG
Word: ##gging | Score: 0.991 | Entity: I-ORG
Word: Face | Score: 0.998 | Entity: I-ORG
Word: Inc | Score: 0.999 | Entity: I-ORG
Word: New | Score: 0.999 | Entity: I-LOC
Word: York | Score: 0.999 | Entity: I-LOC
Word: City | Score: 0.999 | Entity: I-LOC
Word: D | Score: 0.986 | Entity: I-LOC
Word: ##UM | Score: 0.951 | Entity: I-LOC
Word: ##BO | Score: 0.934 | Entity: I-LOC
Word: Manhattan | Score: 0.976 | Entity: I-LOC
Word: Bridge | Score: 0.991 | Entity: I-LOC


In [33]:
res

[{'end': 2,
  'entity': 'I-ORG',
  'index': 1,
  'score': 0.999578595161438,
  'start': 0,
  'word': 'Hu'},
 {'end': 7,
  'entity': 'I-ORG',
  'index': 2,
  'score': 0.9909763932228088,
  'start': 2,
  'word': '##gging'},
 {'end': 12,
  'entity': 'I-ORG',
  'index': 3,
  'score': 0.9982224702835083,
  'start': 8,
  'word': 'Face'},
 {'end': 16,
  'entity': 'I-ORG',
  'index': 4,
  'score': 0.9994880557060242,
  'start': 13,
  'word': 'Inc'},
 {'end': 43,
  'entity': 'I-LOC',
  'index': 11,
  'score': 0.9994344711303711,
  'start': 40,
  'word': 'New'},
 {'end': 48,
  'entity': 'I-LOC',
  'index': 12,
  'score': 0.9993196129798889,
  'start': 44,
  'word': 'York'},
 {'end': 53,
  'entity': 'I-LOC',
  'index': 13,
  'score': 0.9993793964385986,
  'start': 49,
  'word': 'City'},
 {'end': 80,
  'entity': 'I-LOC',
  'index': 19,
  'score': 0.9862582683563232,
  'start': 79,
  'word': 'D'},
 {'end': 82,
  'entity': 'I-LOC',
  'index': 20,
  'score': 0.9514269232749939,
  'start': 80,
  'word

### Summarization 

Summarization is the task of summarizing a document or an article into a shorter text. An example of a summarization dataset is the CNN / Daily Mail dataset, which consists of long news articles and was created for the task of summarization. 

In [34]:
summarizer = pipeline("summarization")

In [35]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

In [37]:
pprint(summarizer(ARTICLE, 
                 max_length=130,
                 min_length=30,
                 do_sample=False))

[{'summary_text': ' Liana Barrientos, 39, is charged with two counts of '
                  '"offering a false instrument for filing in the first '
                  'degree" In total, she has been married 10 times, with nine '
                  'of her marriages occurring between 1999 and 2002 . At one '
                  'time, she was married to eight men at once, prosecutors say '
                  '.'}]
